In [1]:
from transformers import pipeline
from PIL import Image, ImageChops
import requests
import numpy as np
import scipy
import cv2

IMG_PATH = "/Users/herk/Downloads/skydive.jpg"

In [2]:
# load pipe
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")

# load image
image = Image.open(IMG_PATH)

# inference
depth = pipe(image)["depth"]
depth.save("/Users/herk/Downloads/depth.jpg")

In [3]:
def shift_image(img, depth_img, shift_amount=10):
    # Ensure base image has alpha
    img = img.convert("RGBA")
    data = np.array(img)

    # Ensure depth image is grayscale (for single value)
    depth_img = depth_img.convert("L")
    depth_data = np.array(depth_img)
    deltas = np.array((depth_data / 255.0) * float(shift_amount), dtype=int)

    # This creates the transprent resulting image.
    # For now, we're dealing with pixel data.
    shifted_data = np.zeros_like(data)

    width = img.width
    height = img.height

    for y, row in enumerate(deltas):
        width = len(row)
        x = 0
        while x < width:
            dx = row[x]
            if x+dx >= width:
                break
            if x-dx < 0:
                shifted_data[y][x-dx] = [0,0,0,0]
            else:
                shifted_data[y][x-dx] = data[y][x]
            x += 1

    # Convert the pixel data to an image.
    shifted_image = Image.fromarray(shifted_data)

    alphas_image = Image.fromarray(scipy.ndimage.binary_fill_holes(
      ImageChops.invert(
          shifted_image.getchannel("A")
      ))).convert("1")
    shifted_image.putalpha(ImageChops.invert(alphas_image))

    return shifted_image

In [4]:
def shift_and_inpaint(path, amount):
    shifted = shift_image(image, depth, shift_amount=amount).save(path)
    org_img = cv2.imread(filename=path)
    damaged_img = cv2.imread(filename=path)
     
    # get the shape of the image
    height, width = damaged_img.shape[0], damaged_img.shape[1]
     
    # Converting all pixels greater than zero to black while black becomes white
    for i in range(height):
        for j in range(width):
            if damaged_img[i, j].sum() > 0:
                damaged_img[i, j] = 0
            else:
                damaged_img[i, j] = [255, 255, 255]
     
    # saving the mask 
    mask = cv2.cvtColor(damaged_img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("title", mask)
    #cv2.waitKey(0)

    dst = cv2.inpaint(org_img, mask, 3, cv2.INPAINT_NS)
    # Write the output.
    cv2.imwrite(path, dst)

In [5]:
shift_and_inpaint("/Users/herk/Downloads/cart_left.png", 10)
shift_and_inpaint("/Users/herk/Downloads/cart_right.png", 50)

In [6]:

from PIL import Image
import pillow_heif
import math

def load_image(path):
    try:
        return Image.open(path)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def combine_left_and_right():
    left_img_path = "/Users/herk/Downloads/cart_left.png"
    right_img_path = "/Users/herk/Downloads/cart_right.png"
    new_image_path = "/Users/herk/Downloads/cart_with_depth.heic"

    left_img = load_image(left_img_path)
    right_img = load_image(right_img_path)

    print(f"left_img: {left_img}")
    print(f"right_img: {right_img}")

    if left_img is None or right_img is None:
        print("One of the images could not be loaded.")
        return
    
    image_width = left_img.size[0]
    image_height = left_img.size[1]
    fov_horizontal_degrees = 55
    fov_horizontal_radians = fov_horizontal_degrees * (3.14159 / 180)
    focal_length_pixels = 0.5 * image_width / math.tan(0.5 * fov_horizontal_radians)
    baseline = 65.0 # in millimeters

    # Since pillow_heif does not directly support adding depth information,
    # the following is an approximation that creates a HEIF image with two pictures.
    # Depth information and camera intrinsics are not standard and require
    # specific API support which is not available in pillow_heif as of the last knowledge update.

    left_img_bytes = left_img.tobytes("raw")
    right_img_bytes = right_img.tobytes("raw")

    cv_img = cv2.imread(left_img_path, cv2.IMREAD_UNCHANGED)
    heif_file_1 = pillow_heif.from_bytes(
        mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img)
    )
    cv_img = cv2.imread(right_img_path, cv2.IMREAD_UNCHANGED)
    heif_file_2 = pillow_heif.from_bytes(
        mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img)
    )
    heif_file_1.add_from_bytes( mode="BGRA;16",
        size=(cv_img.shape[1], cv_img.shape[0]),
        data=bytes(cv_img))
    heif_file.write_to_file(new_image_path)

combine_left_and_right()

left_img: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=669x446 at 0x2F2E9E4D0>
right_img: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=669x446 at 0x2F2E9EC50>


AttributeError: 'HeifFile' object has no attribute 'add_from_bytes'

In [39]:
read = pillow_heif.read_heif("/Users/herk/Downloads/cart_with_depth.heic")

<HeifFile with 2 images: ['<HeifImage 669x446 RGB with 895122 bytes image data and 0 thumbnails>', '<HeifImage 669x446 RGB with 895122 bytes image data and 0 thumbnails>']>

In [44]:
pillow_heif.HeifFile()

<HeifFile with 0 images: []>

In [16]:
import pyheif

heif_file = pyheif.read('/Users/herk/Downloads/cart_with_depth.heic')

error: struct heif_decoding_options: wrong total size (cdef says 48, but C compiler says 72). fix it or use "...;" as the last field in the cdef for struct heif_decoding_options to make it flexible

In [29]:
image = Image.open('/Users/herk/Downloads/IMG_0012.HEIC')

In [30]:
read = pillow_heif.read_heif("/Users/herk/Downloads/IMG_0012.HEIC")

In [41]:
read.info

{'primary': True,
 'bit_depth': 8,
 'exif': b"Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x06\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x00V\x01\x10\x00\x02\x00\x00\x00\x11\x00\x00\x00\\\x011\x00\x02\x00\x00\x00\x06\x00\x00\x00n\x012\x00\x02\x00\x00\x00\x17\x00\x00\x00t\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\x8c\x88%\x00\x04\x00\x00\x00\x01\x00\x00\x01\xcc\x00\x00\x00\x00Apple\x00Apple Vision Pro\x00\x001.0.2\x002024-02-13T11:22:22PST\x00\x00\x00\r\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x01.\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x016\x883\x00\x04\x00\x00\x00\x01\x00\x00\x00W\x92\x01\x00\n\x00\x00\x00\x01\x00\x00\x01>\x92\x02\x00\x05\x00\x00\x00\x01\x00\x00\x01F\x92\x04\x00\n\x00\x00\x00\x01\x00\x00\x01N\x92\n\x00\x05\x00\x00\x00\x01\x00\x00\x01V\x92|\x00\x07\x00\x00\x008\x00\x00\x01^\xa0\x02\x00\x04\x00\x00\x00\x01\x00\x00\n\x00\xa0\x03\x00\x04\x00\x00\x00\x01\x00\x00\n\x00\xa4\x05\x00\x03\x00\x00\x00\x01\x00\x12\x00\x00\xa43\x00\x02\x00\x00\x00\x06\x00\x00\x01\x96\xa44\x00\x02\x00\x00\